<a href="https://colab.research.google.com/github/Ffranco08/SPIS_Final_FF_RD/blob/main/Song_Reccommendatorinator_(SPIS_Final)_Spotify.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Creating Spotify Developer Credentials

In [25]:
'''
API = application programming interface (borrowing software from other systems, don't need to understand the software, just know what is outputted)
auth – An access token (optional)
requests_session – A Requests session object or a truthy value to create one. A falsy value disables sessions. It should generally be a good idea to keep sessions enabled for performance reasons (connection pooling).
client_credentials_manager – SpotifyClientCredentials object
oauth_manager – SpotifyOAuth object
auth_manager – SpotifyOauth, SpotifyClientCredentials, or SpotifyImplicitGrant object
proxies – Definition of proxies (optional). See Requests doc https://2.python-requests.org/en/master/user/advanced/#proxies
requests_timeout – Tell Requests to stop waiting for a response after a given number of seconds
status_forcelist – Tell requests what type of status codes retries should occur on
retries – Total number of retries to allow
status_retries – Number of times to retry on bad status codes
backoff_factor – A backoff factor to apply between attempts after the second try See urllib3 https://urllib3.readthedocs.io/en/latest/reference/urllib3.util.html
language – The language parameter advertises what language the user prefers to see. See ISO-639-1 language code:
'''


# The Client ID is a public identifier of your application for spotify
CLIENT_ID = '1b5ed5df60f344b48ff2ed1140ec75e8'
# The Server Secret is used to authenticate your application and make requests to APIs
CLIENT_SECRET = 'd31cc8fd2b544844a97af34f77c51409'

# OG CLIENT_ID = '6d32914a82f04c2fb6e59683345918a7'
# OG CLIENT_SECRET ='59ebe333b3234b05b74b977dc787e80d'

###Import the libraries

In [26]:
import numpy as np
import pandas as pd
import plotly.express as px
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import re

###Set up Spotipy

In [27]:
# We will use the Spotipy package to get data from Spotify,
# which we need to install and authenticate before we can use it.

# install for local/offline work. If the environment doesn't have the library, must install it and then import it
!pip install spotipy
import spotipy
# used to autheticate tokens
from spotipy.oauth2 import SpotifyClientCredentials
#Get a token if you want to access your own data - will expire in an hour (allows access to ur data)
auth_manager = SpotifyClientCredentials(client_id=CLIENT_ID, client_secret=CLIENT_SECRET)
sp = spotipy.Spotify(auth_manager=auth_manager)

###Helper functions

To make some of the Spotipy tasks a bit easier, we've defined some helper functions.

In [28]:
# get the audio features of a given list of song objects
def get_audio_features(song_objects):
    #print("Song List", song_objects)
    audio_features = list()
    for song_object in song_objects:
        #print('Individual Item in list:', song_object)
        # new dictionary
        song_audio_features = {
            'title': song_object['name'],
            'artist': song_object['artists'][0]['name']
            #'genre': song_object[artists]],
        }
        # audio features returns a list of dictionary value
        # [{'danceability': 0.42, 'energy': 0.66, 'key': 4, 'loudness': -7.726, 'mode': 0, 'speechiness': 0.326, 'acousticness': 0.016, 'instrumentalness': 0.00284, ... 'duration_ms': 145115, 'time_signature': 4}]
        # accessing the only thing in the list with [0] to extract only the dictionary
        data = sp.audio_features(song_object['id'])[0]

        # goes thru each key in the dictionary (data)
        for k in data:
            # if keys match any of the items in the set, don't add to new dictionary
            if k not in {'analysis_url', 'id', 'track_href', 'type', 'uri'}:
                # adds to the new dictionary
                song_audio_features[k] = data[k]
        audio_features.append(song_audio_features)
    return pd.DataFrame.from_records(audio_features)

# search for a song given a title, artist, and number of desired search results
def search_song(title, artist, num_results=1):
    search_query = 'track:%s artist:%s' % (title, artist) # does % represent var and it is equal to (title, artist)?
    results = sp.search(q=search_query, type='track', limit=num_results)
    #print(results['tracks']['items'])
    return get_audio_features([song for song in results['tracks']['items']])



# get a list of song objects from a playlist URL ()
def get_songs_from_playlist(playlist_url):
    # list of dictionary items that has a key 'track'
    data = sp.playlist(playlist_url.split('/')[-1])['tracks']['items']
    # song_objects = []
    # for curr in data:
    #   song_objects.append(curr['track'])
    # return get_audio_features(song_objects)
    return get_audio_features([curr['track'] for curr in data])

In [29]:
# test_song = search_song('Stargazing', 'Young Cardi')
# test_song.head()

To run the following cell you must have the same csv we grabbed from Kaggle. Here is a Link: https://www.kaggle.com/datasets/byomokeshsenapati/spotify-song-attributes


In [30]:
# Kaggle Data Set
Kaggle_df = pd.read_csv("Spotify_Song_Attributes.csv")
# Kaggle_df = pd.Dataframe(Kaggle_df)
Kaggle_df = Kaggle_df.drop(columns=['type','id', 'uri','track_href','analysis_url'])


# rankings_pd.rename(columns = {'test':'TEST'}, inplace = True)
Kaggle_df.rename(columns = {'trackName': 'title', 'artistName' : 'artist'}, inplace = True)
Kaggle_df.dropna(inplace = True)
Kaggle_df

#feature_columns = ['danceability', 'energy', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'time_signature']

,title,artist,msPlayed,genre,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
1,"""In The Hall Of The Mountain King"" from Peer G...",London Symphony Orchestra,1806234,british orchestra,0.475,0.130,7.0,-17.719,1.0,0.0510,0.9160,0.956000,0.1010,0.122,112.241,150827.0,4.0
2,#BrooklynBloodPop!,SyKo,145610,glitchcore,0.691,0.814,1.0,-3.788,0.0,0.1170,0.0164,0.000000,0.3660,0.509,132.012,145611.0,4.0
3,$10,Good Morning,25058,experimental pop,0.624,0.596,4.0,-9.804,1.0,0.0314,0.4750,0.203000,0.1190,0.896,120.969,89509.0,4.0
4,(I Just) Died In Your Arms,Cutting Crew,5504949,album rock,0.625,0.726,11.0,-11.402,0.0,0.0444,0.0158,0.000169,0.0625,0.507,124.945,280400.0,4.0
5,(L)only Child,salem ilese,2237969,alt z,0.645,0.611,8.0,-5.925,0.0,0.1370,0.2900,0.000021,0.2370,0.645,157.475,144468.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10074,Younger,Ruel,5272303,alt z,0.745,0.477,11.0,-7.706,0.0,0.0880,0.2020,0.000000,0.1200,0.454,136.055,222320.0,4.0
10075,Younger with Time.,Ben Zaidi,668478,folk-pop,0.537,0.143,2.0,-16.992,1.0,0.0331,0.9610,0.005720,0.1100,0.245,131.118,222827.0,3.0
10077,Your Love Is My Drug (8 Bit Slowed),just valery,97600,sad lo-fi,0.282,0.158,6.0,-7.783,1.0,0.0311,0.4380,0.134000,0.4740,0.248,65.152,112582.0,4.0
10078,Your Power,Billie Eilish,988224,art pop,0.632,0.284,9.0,-14.025,0.0,0.0801,0.9320,0.000476,0.2330,0.208,129.642,245897.0,4.0


In [31]:
# user_input = input("Enter song and artist like so...Radioactive, Imagine Dragons. ")


def inputted_song():
  #user_input = input(f"Enter song and artist like so...{'Radioactive, Imagine Dragons'} \n")
  song_input = input('Enter song: ')
  artist_input = input('Enter artist: ')
  return song_input, artist_input


# returns a tuple with the song and artist
song_info = inputted_song()

#print(song_info[0])

# result = sp.search("AJR")
# track = result['tracks']['items'][0]

# artist = sp.artist(track["artists"][0]["external_urls"]["spotify"])
# print("artist genres:", artist["genres"])

# album = sp.album(track["album"]["external_urls"]["spotify"])
# print("album genres:", album["genres"])

test_song = search_song(str(song_info[0]), str(song_info[1]))
result = sp.search(song_info[0]) #sp.search = query
track = result['tracks']['items'][0]

artist = sp.artist(track["artists"][0]["external_urls"]["spotify"])
print("artist genres:", artist["genres"]) #prints list of genres related to song


test_song_df = pd.DataFrame(test_song)
#creating a list which will be converted into a column
genre = artist["genres"]
n = (len(genre))

# #["genre"] = name of column and we set that = to our newly created list which will be the value assigned to it
if len(genre) > 1:
  test_song_df = test_song_df.loc[test_song_df.index.repeat(n)]
test_song_df["genre"] = genre
test_song_df
test_song_df.head()

# if 'Ankit' in df.values :
#     print("\nThis value exists in Dataframe")

# else :
#     print("\nThis value does not exists in Dataframe")


#SOMETIMES RETURNS GENRES ONLY AND NOT ATTRIBUTES RELATED TO SONG BECAUSE SPOITFY DOESN'T HAVE THEM, CHOOSE ANOTHER SONG
#test case = Radioactive | Imagine Dragons

Enter song: Radioactive
Enter artist: Imagine Dragons
artist genres: ['modern rock', 'pop', 'rock']


,title,artist,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,genre
0,Radioactive,Imagine Dragons,0.448,0.784,9,-3.686,1,0.0627,0.106,0.000108,0.668,0.236,136.245,186813,4,modern rock
0,Radioactive,Imagine Dragons,0.448,0.784,9,-3.686,1,0.0627,0.106,0.000108,0.668,0.236,136.245,186813,4,pop
0,Radioactive,Imagine Dragons,0.448,0.784,9,-3.686,1,0.0627,0.106,0.000108,0.668,0.236,136.245,186813,4,rock


You are going to need an the all_songs.csv. Here is a link to the spreadsheet which can be turned into a .csv: https://1drv.ms/x/s!Antcp-1TMi1Cux3z-gEuEkOhE6_0?e=koNAI5

In [32]:
# concatenate playlist dataframes (pd.read_csv turns var into dataframe automatically)

all_songs_raw = pd.read_csv('all_songs.csv')
# inplace = True basically updates all_songs_raw to have removed columns
#all_songs_raw.drop(columns = ['trackName','artistName','msPlayed'], inplace = True)

data_base = pd.concat([all_songs_raw,Kaggle_df])
data_base.to_csv('data_base.csv',index = False)

# when we cocat test_song to data_base we add it to the end of data_base, acess this test song by using all_songs.tail(1)
# all_songs.csv should be only .csv we have to acess, all data is stored on this except for input
# NOT IN VS IN
#if song_info[0] in data_base.values:
#all_songs = data_base[data_base.title != song_info[0]]



all_songs = pd.concat([data_base, pd.DataFrame(test_song).head()])
all_songs.tail()
# drops unnecessary columns
all_songs.drop(columns = ['Unnamed: 0'], inplace = True)
all_songs.drop_duplicates(inplace = True)
all_songs.drop(columns = ['msPlayed'], inplace = True)
#df['DataFrame Column'] = df['DataFrame Column'].fillna(0)
all_songs = all_songs.fillna(0)
d_type = all_songs['genre'].dtypes
# list = object btw
print(d_type)
# Series.values.tolist()
#col_list = all_songs.genre.values.tolist(), don't need to use this since column is already an object



all_songs
# all_songs = all_songs.drop_duplicates()
# print(all_songs['title'].value_counts())
# all_songs.tail()
# all songs basically encompasses all the playlists from playlist_urls

object


,title,artist,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,genre,projections,cos_sim
0,Rich Men North of Richmond,Oliver Anthony Music,0.678,0.264,10.0,-12.384,1.0,0.0622,0.753,0.000000,0.3020,0.528,121.722,187244.0,4.0,Country,[5.587558 4.6795244 3.9695394],0.931992
1,Last Night,Morgan Wallen,0.517,0.675,6.0,-5.382,1.0,0.0357,0.459,0.000000,0.1510,0.518,203.853,163855.0,4.0,Country,[4.909779 5.580968 4.8923483],0.976602
2,Fast Car,Luke Combs,0.712,0.603,8.0,-5.520,1.0,0.0262,0.186,0.000000,0.1150,0.670,97.994,265493.0,4.0,Country,[4.6073217 5.204994 6.971702 ],0.981800
3,Something in the Orange,Zach Bryan,0.369,0.192,4.0,-12.151,0.0,0.0400,0.555,0.000008,0.0954,0.148,175.212,228013.0,3.0,Country,[15.850701 12.677414 0.9390475],0.733115
4,You Proof,Morgan Wallen,0.731,0.850,9.0,-4.913,1.0,0.0336,0.252,0.000000,0.6050,0.644,119.706,157478.0,4.0,Country,[1.8173681 7.135653 7.0070276],0.981680
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5035,Younger with Time.,Ben Zaidi,0.537,0.143,2.0,-16.992,1.0,0.0331,0.961,0.005720,0.1100,0.245,131.118,222827.0,3.0,folk-pop,0,0.000000
5037,Your Love Is My Drug (8 Bit Slowed),just valery,0.282,0.158,6.0,-7.783,1.0,0.0311,0.438,0.134000,0.4740,0.248,65.152,112582.0,4.0,sad lo-fi,0,0.000000
5038,Your Power,Billie Eilish,0.632,0.284,9.0,-14.025,0.0,0.0801,0.932,0.000476,0.2330,0.208,129.642,245897.0,4.0,art pop,0,0.000000
5039,"Your Voice / Bethel, NY",Jaden,0.560,0.344,3.0,-12.283,1.0,0.0306,0.866,0.001470,0.1110,0.428,115.393,213627.0,3.0,pop rap,0,0.000000


###Saving dataframe to CSV file

In [33]:
# Updated all_songs .csv
all_songs.to_csv("all_songs.csv")

In [34]:
# Updated all_songs var
all_songs = pd.read_csv("all_songs.csv")

In [35]:
# Download the file: all_songs_genre.csv
# ! gdown 1n_qzfs0QKtI5RUuigs-9SAr8Nr4m6Rx0

# load into dataframe
# all_songs_genre = pd.read_csv("all_songs_genre.csv")

###Visualizing songs from multiple playlists

In [36]:
# specify the important song features
# we arbitrarily chose these ones, but you should explore the data yourself to see which are most interesting to you
#feature_columns = ['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature']
feature_columns = ['danceability', 'energy', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'time_signature']

In [37]:
# # plot 2D scatterplot matrix
# fig = px.scatter_matrix(
#     all_songs,
#     dimensions = feature_columns,
#     color="genre"
# )
# fig.update_traces(diagonal_visible=False)
# fig.show()

####UMAP


In [38]:
# We will use the umap-learn package to do UMAP dimensionality reduction.
!pip install umap-learn
from umap.umap_ import UMAP
from scipy.spatial import KDTree
import sklearn

In [39]:


#  defining umap
# Uniform Manifold Approximation and Projection
umap = UMAP(n_components=3, init='random', random_state=0)
# fit_transform first fits the UMAP model to input_data, learning the low-dimensional representation.
# Then transforms input_data into the learned low-dimensional space, resulting in umap_embedding (low-dimensional representation of high-dimensional data).
# BASICALLY OUR UMAP IS BEING MADE USING THE FEATURE COLUMNS DATA
projections = umap.fit_transform(all_songs[feature_columns])

all_songs['projections'] = [None] * len(all_songs)

row = 0
for proj in projections:
  # projections in NaN rn for all, row = 0 so ['projections'][row] means start at row 0 for projections column, it is being set = to projections which starts at 1
  all_songs['projections'][row] = proj
  row += 1
# Is this the array of ALL projections (ie. x, y, z, of all songs)?
projections = np.array(projections)

# Song we just inputted is put all the way at the bottom of the dataframe so we go to that location and retrieve its calculated projections
test_song = np.array(all_songs['projections'][len(all_songs) - 1])
#pd.concat([new_row,df.loc[:]]).reset_index(drop=True)

sim_matrix = sklearn.metrics.pairwise.cosine_similarity([test_song], projections, dense_output=True)
all_songs['cos_sim'] = pd.Series(sim_matrix[0])
# #print(all_songs['cos_sim'])
# # from sklearn.neighbors import KNeighborsClassifier
# # Classifier = KNeighborsClassifier(n_neighbors=5)
# # projections4UMAP = umap.fit_transform(all_songs[feature_columns])
# # Classifier.fit(projections4UMAP)
# # print(Classifier.kneighbors(projections4UMAP, 5))
# # fig = px.scatter_3d(projections4UMAP, x=0, y=1, z=2, color=all_songs['genre'])

# # fig.show()

/usr/local/lib/python3.10/dist-packages/umap/umap_.py:1943: UserWarning:

n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.

<ipython-input-39-36bff7841437>:14: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [40]:
fig = px.scatter_3d(projections, x=0, y=1, z=2, color= all_songs['genre'])
fig.show()

In [41]:
test_alls = np.array(all_songs[feature_columns])
test_songg = np.array(test_song_df[feature_columns].iloc[0])

print(test_song)
sim_matrix = sklearn.metrics.pairwise.cosine_similarity([test_songg], test_alls, dense_output=True)
print(sim_matrix)
all_songs['cos_sim'] = pd.Series(sim_matrix[0])


[-1.9212401 13.164271   4.3002086]
[[0.9723911  0.98586072 0.98258417 ... 0.96676041 0.95358182 1.        ]]


In [42]:

inputed_song = all_songs.tail(1)
print(inputed_song)

#excludes first song since that is inputted song (don't wanna reccommend same song they just gave)
new_all_songs = all_songs.sort_values("cos_sim", ascending = False, ignore_index = True)
print(type(new_all_songs))
new_all_songs = new_all_songs.reset_index(drop = True)
new_all_songs.drop(index=new_all_songs.index[0], axis=0, inplace=True)
#new_all_songs.drop(index=new_all_songs.title[song_info[0]], axis=0, inplace=True)
new_all_songs = new_all_songs.drop('Unnamed: 0', axis=1)
#subset specifies which columns we are checking for the same stuff and then drops everything after the first copy
new_all_songs.drop_duplicates(subset=feature_columns, keep = 'first', inplace = True)
#print(np.where(df.City1 == df.City2))
#print(np.where(new_all_songs.title == test_song3.title))
#new_all_songs.drop(index = new_all_songs.index[#var], axis = 0, inplace = True )
#new_all_songs = new_all_songs[new_all_songs['title'] != song_info[0]]
if n > 1:
  cond = new_all_songs['genre'].isin(genre) #checks if genre list is in genre column (True) and returns series of entire dataframe with True and False values
  new_all_songs.drop(new_all_songs[~cond].index, inplace = True) # dropping True columns but we want to keep rows with genre list so ~cond changes True to False to preserve them
  # new_all_songs.drop(columns=["0"], inplace = True)
reccomended_songs = new_all_songs.iloc[1:11]
reccomended_songs_df = pd.DataFrame(reccomended_songs)
reccomended_songs_df.head(10)


# columns then rows when trying to extract data from dataframe

      Unnamed: 0        title           artist  danceability  energy  key  \
9487           0  Radioactive  Imagine Dragons         0.448   0.784  9.0   

      loudness  mode  speechiness  acousticness  instrumentalness  liveness  \
9487    -3.686   1.0       0.0627         0.106          0.000108     0.668   

      valence    tempo  duration_ms  time_signature genre  \
9487    0.236  136.245     186813.0             4.0     0   

                             projections  cos_sim  
9487  [-1.9212401, 13.164271, 4.3002086]      1.0  
<class 'pandas.core.frame.DataFrame'>


,title,artist,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,genre,projections,cos_sim
40,For You (With Rita Ora),Liam Payne,0.535,0.800,2.0,-4.569,0.0,0.0342,0.01830,0.000000,0.421,0.175,113.023,242828.0,4.0,pop,"[-0.12473841, 8.861473, 5.9004564]",0.997665
97,Follow You,Imagine Dragons,0.542,0.732,9.0,-5.956,1.0,0.0521,0.00209,0.000079,0.496,0.489,124.912,175644.0,4.0,modern rock,"[1.8326364, 9.943625, 7.3628125]",0.996651
101,The Hype,Twenty One Pilots,0.530,0.816,9.0,-6.149,1.0,0.0503,0.01050,0.000610,0.379,0.371,87.019,265373.0,4.0,modern rock,"[0.2482698, 8.655466, 6.3422384]",0.996568
120,my bad luv,blackbear,0.640,0.655,9.0,-4.684,0.0,0.0414,0.03620,0.000000,0.456,0.389,122.967,183433.0,4.0,pop,"[-1.3032925, 7.682111, 9.442466]",0.996324
191,Chlorine,Twenty One Pilots,0.609,0.674,10.0,-7.388,0.0,0.0548,0.07350,0.060000,0.345,0.315,90.009,324467.0,4.0,modern rock,"[0.49473238, 8.478812, 5.490903]",0.995574
214,Heavydirtysoul,Twenty One Pilots,0.613,0.873,7.0,-6.376,0.0,0.0449,0.00397,0.001110,0.367,0.392,129.989,234813.0,4.0,modern rock,"[-0.28274548, 9.398279, 6.673992]",0.995378
237,Demons,Imagine Dragons,0.505,0.710,3.0,-3.015,1.0,0.0321,0.19000,0.000250,0.329,0.428,89.938,175200.0,4.0,modern rock,"[3.2428634, 9.590831, 6.01959]",0.995130
260,New,Daya,0.715,0.742,0.0,-4.331,1.0,0.0347,0.05200,0.000025,0.364,0.322,106.971,230880.0,4.0,pop,"[0.4799485, 8.456568, 6.0244308]",0.994902
267,Fantasy,Bazzi,0.699,0.668,5.0,-4.272,0.0,0.0336,0.15400,0.000003,0.362,0.314,144.105,148230.0,4.0,pop,"[0.7520136, 8.346303, 5.4181123]",0.994814
331,i dont love me,blackbear,0.666,0.704,1.0,-5.737,0.0,0.0366,0.01060,0.000011,0.348,0.418,139.972,197149.0,4.0,pop,"[0.5645919, 8.335024, 6.4374723]",0.994241
